## 1. Triming Raw Data - Extract Los Angeles metropolitant data only

Note: Raw data is not uploaded to GitHub. Do not run this section without downloading the raw_data folder!

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import chardet

# Set some options for pandas and to have matplotlib show the charts in the notebook
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline


In [2]:
# Import raw data
test_raw = pd.read_csv('raw_data/std_test_raw.txt', index_col=False)
print(test_raw.head())
print(test_raw.shape)
print(test_raw.columns)

   County Code  District Code  School Code  Filler  Test Year  Subgroup ID  \
0            0              0            0     nan       2019            1   
1            0              0            0     nan       2019            3   
2            0              0            0     nan       2019            4   
3            0              0            0     nan       2019            6   
4            0              0            0     nan       2019            7   

  Test Type Total Tested At Entity Level Total Tested with Scores  Grade  ...  \
0         B                      3165580                  3162910      3  ...   
1         B                      1616938                  1615465      3  ...   
2         B                      1548642                  1547445      3  ...   
3         B                      2651676                  2649540      3  ...   
4         B                       132465                   132405      3  ...   

   Area 1 Percentage Below Standard Area 2 P

In [3]:
## Triming data to only include Los Angeles Metro Politant
# Only include data for Los Angeles and Orange County
# Which is based on the County Code, as per the school_location dataset
# County Codes for LA and Orange are 19 and 30

# 1. Check County Code for raw data
print(test_raw['County Code'].unique())
print(test_raw['County Code'].describe())

# 2. Select County Code 19 & 30 only
test_raw = test_raw.copy()[(test_raw['County Code'] == 19) | 
                               (test_raw['County Code'] == 30)]
print(test_raw.tail())
print(test_raw.shape)
print(test_raw['County Code'].value_counts())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58]
count   3,576,491.00
mean           29.22
std            14.25
min             0.00
25%            19.00
50%            31.00
75%            39.00
max            58.00
Name: County Code, dtype: float64
         County Code  District Code  School Code  Filler  Test Year  \
1770622           30          76893       130765     nan       2019   
1770623           30          76893       130765     nan       2019   
1770624           30          76893       130765     nan       2019   
1770625           30          76893       130765     nan       2019   
1770626           30          76893       130765     nan       2019   

         Subgroup ID Test Type Total Tested At Entity Level  \
1770622          222         B                            *   
1770623          223         B                            *   
1

In [4]:
## Triming data to only include test score for all students only (Subgroup ID = 1)
# Excluding test scores for detailed demographics (i.e. by gender, by ethnicity, etc.)
test_raw = test_raw.copy()[test_raw['Subgroup ID'] == 1]
print(test_raw.iloc[:10,:14])
print(test_raw.shape)

        County Code  District Code  School Code  Filler  Test Year  \
664554           19              0            0     nan       2019   
664605           19              0            0     nan       2019   
664656           19              0            0     nan       2019   
664707           19              0            0     nan       2019   
664758           19              0            0     nan       2019   
664809           19              0            0     nan       2019   
664860           19              0            0     nan       2019   
664911           19              0            0     nan       2019   
664962           19              0            0     nan       2019   
665013           19              0            0     nan       2019   

        Subgroup ID Test Type Total Tested At Entity Level  \
664554            1         B                       743639   
664605            1         B                       743639   
664656            1         B              

In [5]:
## Triming data to only include test score for schools only, not include district or county level
# which is excluding all entries has School Code is 0 
test_raw = test_raw.copy()[test_raw['School Code'] != 0]
print(test_raw.iloc[:5,:5])
print(test_raw.shape)
print(test_raw['Mean Scale Score'].describe())

        County Code  District Code  School Code  Filler  Test Year
665834           19          10199       100776     nan       2019
665868           19          10199       100776     nan       2019
665902           19          10199       100776     nan       2019
665936           19          10199       100776     nan       2019
665967           19          10199       100776     nan       2019
(23486, 32)
count     17740
unique     3002
top           *
freq       1346
Name: Mean Scale Score, dtype: object


In [6]:
## Triming data to exclude NaN values in the 'Mean Scale Score' column (standard test score)
test_raw = test_raw.copy()[~test_raw['Mean Scale Score'].isin(['*', np.NaN])]
print(test_raw.iloc[:10,:14])
print(test_raw.shape)
print(test_raw['Mean Scale Score'].describe())

        County Code  District Code  School Code  Filler  Test Year  \
665834           19          10199       100776     nan       2019   
665868           19          10199       100776     nan       2019   
665902           19          10199       100776     nan       2019   
665936           19          10199       100776     nan       2019   
666009           19          10199       100776     nan       2019   
666043           19          10199       100776     nan       2019   
666077           19          10199       100776     nan       2019   
666111           19          10199       100776     nan       2019   
666184           19          10199       109660     nan       2019   
666209           19          10199       109660     nan       2019   

        Subgroup ID Test Type Total Tested At Entity Level  \
665834            1         B                          358   
665868            1         B                          358   
665902            1         B              

In [7]:
# Export pre-processing data to csv
test_raw.to_csv('data/3_std_test_scores.csv', index=False)

## 2. Process Data - Calculate test API

In [8]:
# Import data from trimmed data
test = pd.read_csv('data/3_std_test_scores.csv', index_col=False)
print(test.iloc[:5,:14])
print(test.shape)
print(test.Grade.value_counts())
print(test.columns)

   County Code  District Code  School Code  Filler  Test Year  Subgroup ID  \
0           19          10199       100776     nan       2019            1   
1           19          10199       100776     nan       2019            1   
2           19          10199       100776     nan       2019            1   
3           19          10199       100776     nan       2019            1   
4           19          10199       100776     nan       2019            1   

  Test Type  Total Tested At Entity Level  Total Tested with Scores  Grade  \
0         B                           358                       358      6   
1         B                           358                       358      7   
2         B                           358                       358      8   
3         B                           358                       358     11   
4         B                           355                       355      6   

   Test Id  CAASPP Reported Enrollment  Students Tested  Mean 

In [9]:
# Dropped the not needed data
test = test.drop(['County Code','Filler','Test Year', 'Subgroup ID', 'Test Type',
         'Total Tested At Entity Level','CAASPP Reported Enrollment',
          'Percentage Standard Exceeded', 'Percentage Standard Met',
          'Percentage Standard Met and Above', 'Percentage Standard Nearly Met',
          'Percentage Standard Not Met', 'Students with Scores',
          'Area 1 Percentage Above Standard', 'Area 1 Percentage Near Standard',
          'Area 1 Percentage Below Standard', 'Area 2 Percentage Above Standard',
          'Area 2 Percentage Near Standard', 'Area 2 Percentage Below Standard',
          'Area 3 Percentage Above Standard', 'Area 3 Percentage Near Standard',
          'Area 3 Percentage Below Standard', 'Area 4 Percentage Above Standard',
          'Area 4 Percentage Near Standard', 'Area 4 Percentage Below Standard'], axis=1)
test.columns = ['districtcode','schoolcode','totalstudents','grade','test_id', 
                'students_tested','testscore']
test.head()

,districtcode,schoolcode,totalstudents,grade,test_id,students_tested,testscore
0,10199,100776,358,6,1,70,"2,413.60"
1,10199,100776,358,7,1,114,"2,416.50"
2,10199,100776,358,8,1,112,"2,443.20"
3,10199,100776,358,11,1,62,"2,526.30"
4,10199,100776,355,6,2,69,"2,398.90"


In [10]:
## Import API definition file
api = pd.read_csv('data/10_assign_api.csv', index_col=False)
api

,category,group_API,grade3_eng,grade3_math,grade4_eng,grade4_math,grade5_eng,grade5_math,grade6_eng,grade6_math,grade7_eng,grade7_math,grade8_eng,grade8_math,grade11_eng,grade11_math,graduation_rate,college_rate
0,Below Average,1,2114,2189,2131,2204,2201,2219,2210,2235,2258,2250,2288,2265,2299,2280,0,0
1,Below Average,2,2198,2253,2226,2273,2281,2298,2292,2314,2332,2328,2354,2345,2364,2368,40,20
2,Below Average,3,2283,2317,2321,2342,2362,2376,2375,2394,2405,2406,2421,2424,2428,2455,70,35
3,Nearly met,4,2367,2381,2416,2411,2442,2455,2457,2473,2479,2484,2487,2504,2493,2543,83,50
4,Nearly met,5,2400,2409,2445,2448,2472,2492,2494,2513,2516,2526,2527,2545,2538,2586,93,60
5,Standard met,6,2432,2436,2473,2485,2502,2528,2531,2552,2552,2567,2567,2586,2583,2628,94,70
6,Standard met,7,2447,2452,2488,2501,2522,2541,2553,2567,2576,2584,2592,2603,2608,2651,95,75
7,Standard met,8,2461,2469,2503,2517,2542,2554,2575,2581,2601,2601,2618,2620,2633,2673,96,80
8,Standard Exceed,9,2476,2485,2518,2533,2562,2566,2596,2596,2625,2618,2643,2636,2657,2696,98,85
9,Standard Exceed,10,2490,2501,2533,2549,2582,2579,2618,2610,2649,2635,2668,2653,2682,2718,99,90


In [11]:
## Calculate API for each grade and test subject
test['test_API']= ''
grades = [3, 4, 5, 6, 7, 8, 11]

# Looping through each grade and each test subject
for j in grades: 
    for i in [1,2]:  # Test subject 1 is english, 2 is math
        filter = (test.grade == j) & (test.test_id == i)
        labels = list(range(1,11))
        # Choose bins base on grade & subject
        if j == 3 and i == 1:
            bins = list(api.iloc[:,2]) # grade3_eng cut score
        elif j == 3 and i == 2:
            bins = list(api.iloc[:,3]) # grade3_math cut score
        elif j == 4 and i == 1:
            bins = list(api.iloc[:,4]) # grade4_eng cut score
        elif j == 4 and i == 2:
            bins = list(api.iloc[:,5]) # grade4_math cut score        
        elif j == 5 and i == 1:
            bins = list(api.iloc[:,6]) # grade5_eng cut score
        elif j == 5 and i == 2:
            bins = list(api.iloc[:,7]) # grade5_math cut score 
        elif j == 6 and i == 1:
            bins = list(api.iloc[:,8]) # grade6_eng cut score
        elif j == 6 and i == 2:
            bins = list(api.iloc[:,9]) # grade6_math cut score             
        elif j == 7 and i == 1:
            bins = list(api.iloc[:,10]) # grade7_eng cut score
        elif j == 7 and i == 2:
            bins = list(api.iloc[:,11]) # grade7_math cut score
        elif j == 8 and i == 1:
            bins = list(api.iloc[:,12]) # grade8_eng cut score
        elif j == 8 and i == 2:
            bins = list(api.iloc[:,13]) # grade8_math cut score
        elif j == 11 and i == 1:
            bins = list(api.iloc[:,14]) # grade11_eng cut score
        elif j == 11 and i == 2:
            bins = list(api.iloc[:,15]) # grade11_math cut score
        
        # Apply cut function
        test.loc[filter,['test_API']] = pd.cut(test.testscore, bins=bins,\
                                               labels=labels, include_lowest=True)
        
test

,districtcode,schoolcode,totalstudents,grade,test_id,students_tested,testscore,test_API
0,10199,100776,358,6,1,70,"2,413.60",3
1,10199,100776,358,7,1,114,"2,416.50",3
2,10199,100776,358,8,1,112,"2,443.20",3
3,10199,100776,358,11,1,62,"2,526.30",4
4,10199,100776,355,6,2,69,"2,398.90",3
...,...,...,...,...,...,...,...,...
16389,76893,130765,374,5,2,64,"2,469.00",4
16390,76893,130765,374,6,2,54,"2,482.40",4
16391,76893,130765,374,7,2,51,"2,481.60",3
16392,76893,130765,374,8,2,65,"2,520.40",4


In [12]:
# Add school type for grouping & calculate API elemetary, middle, high school
bins = [1,5,8,12]
labels = ['elementary','middle','high']
test['schooltype'] = pd.cut(test.grade, bins=bins, labels=labels)
test

,districtcode,schoolcode,totalstudents,grade,test_id,students_tested,testscore,test_API,schooltype
0,10199,100776,358,6,1,70,"2,413.60",3,middle
1,10199,100776,358,7,1,114,"2,416.50",3,middle
2,10199,100776,358,8,1,112,"2,443.20",3,middle
3,10199,100776,358,11,1,62,"2,526.30",4,high
4,10199,100776,355,6,2,69,"2,398.90",3,middle
...,...,...,...,...,...,...,...,...,...
16389,76893,130765,374,5,2,64,"2,469.00",4,elementary
16390,76893,130765,374,6,2,54,"2,482.40",4,middle
16391,76893,130765,374,7,2,51,"2,481.60",3,middle
16392,76893,130765,374,8,2,65,"2,520.40",4,middle


In [13]:
# Group grades together by school type
test = test.astype({'students_tested':'float', 'testscore':'float','test_API':'int'})

test_grouped = test.groupby(['schoolcode','schooltype', 'test_id'])

# use max test score since it is the test score of the highest grade (i.e. grade5 in elementary, 8 in middle)
test2 = test_grouped.agg({'students_tested':'sum', 'testscore': 'max','test_API':'mean'}).dropna().unstack()
test2.columns = ['student_tested1','student_tested2','testscore1','testscore2','test_API1','test_API2']
test2 = test2.reset_index()
test2

,schoolcode,schooltype,student_tested1,student_tested2,testscore1,testscore2,test_API1,test_API2
0,100081,high,608.00,608.00,"2,568.30","2,508.80",5.00,3.00
1,100289,elementary,176.00,175.00,"2,517.70","2,534.10",5.00,5.67
2,100354,high,127.00,127.00,"2,710.60","2,679.70",10.00,8.00
3,100602,high,137.00,137.00,"2,645.60","2,631.80",8.00,6.00
4,100610,elementary,293.00,293.00,"2,512.70","2,491.50",5.33,5.00
...,...,...,...,...,...,...,...,...
3595,6120679,elementary,221.00,222.00,"2,429.60","2,416.50",3.33,3.67
3596,6120810,elementary,227.00,235.00,"2,459.10","2,456.80",3.67,4.00
3597,6120877,elementary,349.00,347.00,"2,484.00","2,491.80",5.00,4.67
3598,6120877,middle,242.00,241.00,"2,593.70","2,599.80",6.33,5.67


In [14]:
## Merge with Zip code, school name, district name information
# Import school data
school_raw = pd.read_csv('data/8_public_school.csv', index_col=False)
# print(school_raw.head())

# Trim data, only include columns of interest
school = pd.DataFrame([school_raw['School Code'], school_raw['School Name'],\
                       school_raw['District Code'],school_raw['District Name'],\
                       school_raw['Zip Code']]).T
school.columns = ['schoolcode', 'schoolname','districtcode','districtname','zip']
school


,schoolcode,schoolname,districtcode,districtname,zip
0,100776,North Valley Military Institute College Prepar...,10199,North Valley Military Institute College,91352
1,109660,Aspire Antonio Maria Lugo Academy,10199,Aspire Antonio Maria Lugo Academy,90255
2,109942,Los Angeles International Charter High,10199,Los Angeles International Charter High,90042
3,112128,Aspire Ollin University Preparatory Academy,10199,Aspire Ollin University Preparatory Acad,90255
4,115030,Magnolia Science Academy 3,10199,Magnolia Science Academy 3,90746
...,...,...,...,...,...
2749,6029086,Rossmoor Elementary,73924,Los Alamitos Unified,90720
2750,6030514,J. H. McGaugh Elementary,73924,Los Alamitos Unified,90740
2751,6058853,Oak Middle,73924,Los Alamitos Unified,90720
2752,6058879,Sharon Christa McAuliffe Middle,73924,Los Alamitos Unified,90720


In [15]:
# Merge test score and school zip code
test2 = school.merge(test2, on='schoolcode')

# Export to csv for merging with other dataset
test2.to_csv('data/11_test_merged.csv', index=False)

test2

,schoolcode,schoolname,districtcode,districtname,zip,schooltype,student_tested1,student_tested2,testscore1,testscore2,test_API1,test_API2
0,100776,North Valley Military Institute College Prepar...,10199,North Valley Military Institute College,91352,middle,296.00,293.00,"2,443.20","2,406.50",3.00,2.67
1,100776,North Valley Military Institute College Prepar...,10199,North Valley Military Institute College,91352,high,62.00,62.00,"2,526.30","2,491.20",4.00,3.00
2,109660,Aspire Antonio Maria Lugo Academy,10199,Aspire Antonio Maria Lugo Academy,90255,elementary,214.00,214.00,"2,486.40","2,522.00",5.67,6.33
3,109942,Los Angeles International Charter High,10199,Los Angeles International Charter High,90042,high,37.00,35.00,"2,562.70","2,534.90",5.00,3.00
4,112128,Aspire Ollin University Preparatory Academy,10199,Aspire Ollin University Preparatory Acad,90255,middle,195.00,195.00,"2,547.10","2,530.30",4.67,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3490,6058853,Oak Middle,73924,Los Alamitos Unified,90720,middle,"1,174.00","1,173.00","2,654.70","2,621.00",9.00,8.67
3491,6058879,Sharon Christa McAuliffe Middle,73924,Los Alamitos Unified,90720,middle,"1,176.00","1,176.00","2,639.40","2,620.20",8.67,8.33
3492,130765,Magnolia Science Academy Santa Ana,76893,SBE - Magnolia Science Academy Santa Ana,92703,elementary,177.00,177.00,"2,467.10","2,469.00",4.00,4.33
3493,130765,Magnolia Science Academy Santa Ana,76893,SBE - Magnolia Science Academy Santa Ana,92703,middle,170.00,170.00,"2,548.30","2,520.40",4.33,3.67
